In [94]:
import yaml
import pandas as pd

In [95]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [96]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

In [97]:
edge:object = webdriver.Edge(executable_path="./utils/msedgedriver.exe")

### Functions:

In [98]:
def resources(name:str) -> dict[str:str]:
    with open("xpath.yml", "r") as f:
        content:dict[str:str] = yaml.safe_load(f)
        return content[name]

In [99]:
def close_window(driver:object) -> None:
    button:str = "//a[@class='btn btn-subscription-light']"
    driver.find_element(by=By.XPATH, value=button).click()

In [100]:
def choose_category(category:str, driver:object) -> None:
    driver.find_element(by=By.XPATH, value=category).click()

In [101]:
def close_ad(element:object,path:str) -> None:
    button = element.find_element(by=By.XPATH, value=path)
    button.click()

In [102]:
def extract_description(element:object, path:str) -> str:
    description:str = element.find_element(by=By.XPATH, value=path)
    return description

In [103]:
def extract_price(element:object, path:str) -> str:
    price:str = element.find_element(by=By.XPATH, value=path)
    return price

## Extraction 🧲

### Sedan cars:

In [104]:
descriptions:list[str] = []
prices:list[str] = []
models:list[str] = []
# seconds
DELAY = 3

In [105]:
sedan = resources("cars_sedan")

In [106]:
edge.get(sedan["url"])

In [107]:
close_window(edge)

In [108]:
elements = edge.find_elements(by=By.XPATH, value=sedan["ad"])

Extract descriptions

In [109]:
head = edge.find_element(by=By.XPATH, value=sedan["ad"])
head.click()
try:
    for element in range(5):
        dynamic_delay = WebDriverWait(edge, DELAY).until(EC.presence_of_element_located((By.XPATH, sedan["description"])))
        temp = extract_description(edge, sedan["description"])
        descriptions.append(temp.text)
        close_ad(edge, sedan["next"])
except TimeoutException as TE:
    descriptions.append("Web wasn't able to scrape descriptions")
except NoSuchElementException as NSE:
    close_ad(edge, sedan["exit"])

In [118]:
close_ad(edge, sedan["exit"])

Extract prices

In [119]:
try:
    prices = edge.find_elements(by=By.XPATH, value=sedan["price"])
    prices = [price.text for price in prices]
except Exception as e:
    prices.append("Web wasn't able to scrape prices")

Extract models

In [120]:
try:
    models = edge.find_elements(by=By.XPATH, value=sedan["model"])
    models = [model.text for model in models]
except Exception as e:
    models.append("Web wasn't able to scrape prices")

## Procesing ⚙

In [122]:
df = pd.DataFrame(
    {
        "model": models[:5],
        "price": prices[:5],
        "description": descriptions
    }
)

In [123]:
df

,model,price,description
0,Nissan Sentra,"$9,850",Garantía: Como es visto no hay garantía\nFinan...
1,Kia Forte,"$9,000",Garantía: Como es visto no hay garantía\nFinan...
2,Honda City,"$8,000\n(Rebajado 4%)",Financiamiento: no\nPrecio negociable: si\nCol...
3,Toyota Corolla,"$10,800",Financiamiento: no\nPrecio negociable: si\nDES...
4,Daihatsu Rocky,"$5,000",Color : Rojo y Negro\nSeñor Guzmán contacto
